In [13]:
# 这个文件用来检查model1的信号转好功能是否正常
# 这个功能很容易出错

import os
import sys
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

parent_path = os.path.dirname(os.getcwd())
tools_path = os.path.join(parent_path, 'tools')
sys.path.append(parent_path)
sys.path.append(tools_path)

from tools.path import get_env_path, get_prompt_from_file
from tools.CTA_GEP import GeneticProgram
from tools.backtest import BacktestEngine
from tools.more_operators import *
from tools.evaluate_and_visualizations import *

In [14]:
data = pd.read_feather('/Users/syesw/Desktop/Generative_AI_Quant/data/AAPL_with_indicators.feather')

train_size = int(len(data) * 0.8)
train_df = data[:train_size]
test_df = data[train_size:]

In [15]:
#signal_equation = "signals = add(MA(self.df.loc[i, 'BIAS'], self.df.loc[i, 'open']), self.df.loc[i, 'BIAS'])"
signal_equation = "signals = add(self.df.loc[i, 'BIAS'], ts_corr(ts_mean_return(delta(ts_stddev(self.df.loc[i, 'EMA'], self.df.loc[i, 'SMA']), self.df.loc[i, 'SMA']), self.df.loc[i, 'DX']), ts_mean_return(self.df.loc[i, 'low'], MIDPRICE(math.sin(8), self.df.loc[i, 'EMA'], self.df.loc[i, 'RSI'])), self.df.loc[i, 'ATR']))"


In [16]:
bt_train = BacktestEngine(data=train_df, signal_str=signal_equation)
bt_train_return = bt_train.run_backtest()
train_return_rate, train_max_drawdown, train_sharpe_ratio = get_backtest_ratios(bt_train_return)


bt_test = BacktestEngine(data=test_df, signal_str=signal_equation)
bt_test_return = bt_test.run_backtest()
test_return_rate, test_max_drawdown, test_sharpe_ratio = get_backtest_ratios(bt_test_return)

bt = BacktestEngine(data=data, signal_str=signal_equation)
bt_return = bt.run_backtest()

In [17]:
print("test_return_rate:" + str(test_return_rate))
print("test_sharpe_ratio:" + str(test_sharpe_ratio))
print("test_max_drawdown:" + str(test_max_drawdown))

test_return_rate:-9.205828769919911
test_sharpe_ratio:-0.43444422854157816
test_max_drawdown:-17.247722248144633
